# Model Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
df_train = pd.read_csv(path+'train_data.csv')
df_val = pd.read_csv(path+'val_data.csv')
df_test = pd.read_csv(path+'test_data.csv')

In [12]:
# Converting dataframes -> numpy arrays to train models off of 
train = df_train.values
val = df_val.values
test = df_test.values

Xtr = train[:,1:]
Ytr = train[:,0]
Xval = val[:,1:]
Yval = val[:,0]
Xte = test[:,1:]
Yte = test[:,0]

In [20]:
# Normalizing data to use simple Keras models
from sklearn.preprocessing import normalize

ModuleNotFoundError: No module named 'sklearn'

In [18]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=1, activation='linear', input_dim=708))
model.compile(loss='mean_squared_error',
              optimizer='SGD',
              metrics=['accuracy'])
model.fit(Xtr, Ytr, epochs=5, batch_size=128)
loss_and_metrics = model.evaluate(Xtr, Ytr, batch_size=128)

Epoch 1/5
22770/22770 [==============================] - 1s 36us/step - loss: nan - acc: 8.7835e-05
Epoch 2/5
22770/22770 [==============================] - 1s 26us/step - loss: nan - acc: 0.0000e+00
Epoch 3/5
22770/22770 [==============================] - 1s 33us/step - loss: nan - acc: 0.0000e+00
Epoch 4/5
22770/22770 [==============================] - 1s 35us/step - loss: nan - acc: 0.0000e+00
Epoch 5/5
22770/22770 [==============================] - 0s 11us/step
